<a href="https://colab.research.google.com/github/tlokeshkumar1/QA-Bot-with-RAG-Model/blob/main/RAG_Model_for_QA_Bot_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pinecone-client

In [ ]:
import pinecone

# Create a Pinecone instance
pinecone = pinecone.Pinecone(api_key="5e668bf2-1adb-4ec2-bd9f-b72b275d0ce1", environment="us-west1-gcp") # Use Pinecone class to initialize

# Create index (if not already created)
index_name = "heros"

# Connect to the index
index = pinecone.Index(index_name)

In [ ]:
# Load the model
model_name = "sentence-transformers/all-MiniLM-L12-v2"  # Choose a 384-dimensional model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def embed_text(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1).numpy()
    return embeddings

In [ ]:
# Example documents
documents = [
    "Document 1 Mahesh Babu is a prominent Indian film actor and producer known for his work primarily in Telugu cinema. Born on August 9, 1975, in Chennai, Tamil Nadu, he hails from a family deeply rooted in the film industry, as he is the son of veteran actor Krishna and began acting in films at a young age.",
    "Document 2 Nani, born as Naveen Babu Ghanta on February 24, 1984, in Hyderabad, India, is a well-known Indian actor and film producer primarily recognized for his work in Telugu cinema. He began his career as an assistant director and made his acting debut in the film Ashta Chamma in 2008, which earned him critical acclaim and established him as a talented actor.",
    "Document 3 Adivi Sesh is an Indian film actor, writer, and director known for his work in Telugu cinema. Born on December 17, 1985, in Hyderabad, he made his acting debut in Sontham (2002) and gained recognition for his roles in critically acclaimed films like Kshanam (2016) and Goodachari (2018). Sesh is celebrated for his versatile performances and storytelling abilities, having also written and directed Kshanam. His dedication to his craft and choice of challenging roles have established him as a prominent figure in contemporary Indian cinema.",
]

# Store embeddings in Pinecone
for i, doc in enumerate(documents):
    embedding = embed_text(doc)[0]  # Get the first element of the array
    index.upsert([(f'doc-{i}', embedding.tolist())])  # Store embedding in Pinecone

In [ ]:
def query_bot(user_query):
    query_embedding = embed_text(user_query)[0].tolist()
    # Retrieve top 3 relevant documents using keyword arguments
    results = index.query(vector=query_embedding, top_k=3)  # Use keyword arguments
    return [result['id'] for result in results['matches']]

def generate_response(doc_ids, user_query):
    # Retrieve only the relevant document based on the user query
    relevant_context = ""
    query_name = user_query.split("about")[-1].strip().lower()  # Extract and normalize the name from the query

    for doc_id in doc_ids:
        doc_index = int(doc_id.split('-')[1])
        # Normalize the document text to lowercase for comparison
        if query_name in documents[doc_index].lower():
            relevant_context = documents[doc_index]
            break  # Stop once the relevant document is found

    if not relevant_context:  # Fallback if no relevant document was found
        relevant_context = "Sorry, I couldn't find relevant information."

    return f"Based on your query '{user_query}', here's the information: {relevant_context}"

def answer_question(user_query):
    doc_ids = query_bot(user_query)  # Get document IDs
    response = generate_response(doc_ids, user_query)  # Pass user_query here
    return response

In [ ]:
# Example user query
user_query = "give me about sesh"
answer = answer_question(user_query)
print(answer)

Based on your query 'give me about sesh', here's the information: Document 3 Adivi Sesh is an Indian film actor, writer, and director known for his work in Telugu cinema. Born on December 17, 1985, in Hyderabad, he made his acting debut in Sontham (2002) and gained recognition for his roles in critically acclaimed films like Kshanam (2016) and Goodachari (2018). Sesh is celebrated for his versatile performances and storytelling abilities, having also written and directed Kshanam. His dedication to his craft and choice of challenging roles have established him as a prominent figure in contemporary Indian cinema.
